In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools
def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/edges_node_names_explainer_subgraphs.csv")
edges_node_names_explainer_subgraphs

,Unnamed: 0,node1,node2,0
0,0,FPGS8,10fthf5glu_c,0.0
1,1,FPGS8,10fthf6glu_c,0.0
2,2,FPGS8,atp_c,0.0
3,3,FPGS8,adp_c,0.0
4,4,FPGS8,glu_L_c,0.0
...,...,...,...,...
53117,53117,r1947,lys_L_e,0.0
53118,53118,r1947,tyr_L_e,0.0
53119,53119,gp1calpha_hs_e,EX_gp1calpha_hs_e,0.0
53120,53120,gp1calpha_hs_e,GP1CALPHAte,0.0


In [11]:
genes = []


for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    
    
    if node_2 in dict_rxn_to_gene.keys():
        
        genes.append(dict_rxn_to_gene[node_2])
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        genes.append(dict_rxn_to_gene[node_1])
        
    else:
        genes.append([''])
        


results_for_R =  pd.concat([pd.Series(genes, name='genes'), edges_node_names_explainer_subgraphs], axis=1).drop('Unnamed: 0',axis=1).reset_index(drop=True)


In [14]:
results_for_R[results_for_R['0']> 0].to_csv("./results/data/results_for_R_non_zero.csv")

In [13]:
results_for_R.to_csv("./results/data/results_for_R.csv")